In [7]:
import jieba
import numpy as np
import pickle
import pathlib
import re


class Sentiment(object):
    """
    文本情感计算类，支持导入自定义词典

    默认使用知网Hownet词典进行情感分析
        >>> from cnsenti import Sentiment
        >>> senti = Sentiment()

    统计文本中情感词个数，
    返回的pos和neg是词语个数
        >>>test_text= '
1,很快，好吃，味道足，量大
1,没有送水没有送水没有送水
1,非常快，态度好。
1,方便，快捷，味道可口，快递给力
1,菜味道很棒！送餐很及时！
1,今天师傅是不是手抖了，微辣格外辣！
1,"送餐快,态度也特别好,辛苦啦谢谢"
1,超级快就送到了，这么冷的天气骑士们辛苦了。谢谢你们。麻辣香锅依然很好吃。
1,经过上次晚了2小时，这次超级快，20分钟就送到了……
1,最后五分钟订的，卖家特别好接单了，谢谢。
1,量大，好吃，每次点的都够吃两次
1,挺辣的，吃着还可以吧
1,味道好，送餐快，分量足
1,量足，好吃，送餐也快
1,特别好吃，量特大，而且送餐特别快，特别特别棒
1,口感好的很，速度快！
1,相当好吃的香锅，分量够足，味道也没的说。
1,好吃！速度！包装也有品质，不出家门就能吃到餐厅的味道！
1,味道好极啦，送餐很快师傅辛苦啦
1,量大味道好，送餐师傅都很好
1,送餐师傅很好，味道好极啦
1,送货速度很快，一直定这家，赞
1,很方便，很快就送到了。棒
1,好吃，总点，这么多够五个人吃。送的很快。
1,"很香很美味,下次还会光顾"
1,"送餐特别快,态度也好,辛苦啦"
1,服务很不错，送到的很快，半小时不到就送来了
1,速度很快，大雾霾天外卖骑士态度都很好，赞赞赞！
1,味道正宗，量大内容多
1,"送餐非常快,态度特别好,谢谢"
1,又快又好，量足，经常吃
1,好大一盆点了7个小份量足
1,配送人员态度好，速度快！
1,"在这种天气里感谢送餐员的辛苦服务,谢谢啦"
1,"送餐特别快,态度好,非常感谢"
1,送的非常快，包装好！谢谢师傅！
1,附近最好吃的麻辣香锅，不开玩笑的
1,味道不错，份量很足，建议都点小份。红薯超好吃就是太烂了容易碎
1,还不错，就是稍微咸了点
1,这么晚辛苦外卖小哥了
1,超级快就送到了，谢谢骑士很快，感谢骑士这种天气还在工作！
1,非常好吃，味道也很香，推荐！
1,"很好吃,速递快,下次继续选择"
1,很快，特别好
1,太麻了，青笋有点小，米饭给的也不多，土豆片都碎了，找不到了
1,点了太多次了，味道很香
1,"态度很好,地址填错了还是给我跑了一趟,没有表现出不愿意的样子,为了这个快递员,我写了评论"
1,快递小哥很快就送到了！赞！水煮牛肉肉质鲜嫩，辣的恰到好处，也很入味。不错，挺好吃的！
1,"口味,不错，干净味道好，送货员服务非常好！"
1,"送餐特别快,态度好,辛苦啦,谢谢"
1,"送餐快,送餐大哥态度好,辛苦啦"
1,"送餐很快,态度也很好,谢谢"
1,挺不错的！
1,快递大叔人特别好。就是百度外卖现在连个优惠都那个少，真要投奔饿了么了。。。。
1,感觉没有在店里的好吃，感谢送餐师傅我们家六楼没电梯还得爬楼梯
1,菜场好，挺好吃的
1,快递小哥辛苦了！下雨天的真是辛苦您了！给你点赞！
1,相当好！送餐快，一大盒子超好吃！
1,"送餐特别快,送餐员态度也特别好"
1,速度快，量多，很香
1,味道很正点！餐具很好用！送餐速度快！
1,鱼豆腐有异味儿，有点腻。
1,非常好！第一次用百度外卖，送餐很快，快递小哥也很礼貌。菜品量足，味道不错，下次再来。
1,味道不错！
1,东西还可以，但是没有在店里吃的好吃
1,锅底怎么是13？
1,"很香,很好吃"
1,"很好,送餐员也很好"
1,菜量给的挺足的，很满意，就是墨鱼丸不太好，有股臭味，其他都很好吃。
1,不错好吃，就是最后有点油
1,一个小时送到的，很好吃，但是点了平菇怎么没给
1,辣椒放太多了吧
1,锅仔酸菜炖腔骨怎么没见酸菜啊？只有青笋和腔骨，青笋也不是酸的。
1,很合口味。女友夸猪蹄好吃。过年期间都不容易。给快递小哥个赞。辛苦。
1,炒饭有点儿油，不过菜还是很好的
1,送餐很及时，菜品味道不错！
1,饭还挺实在给个好评送餐挺快
1,骑士服务态度很好，速度很迅速！
1,最近一直订这家的外卖，相当不错，够味儿，快递小哥也是棒棒的！
1,量很多，就是尖椒豆腐皮的味道差点，不是尖椒，是大海椒，一点辣味都没有呢！
1,味道还不错，红烧肉更软点就更完美啦！
1,"快递员很好,又快又负责"
1,味道很好，送餐速度也很快。
1,一直在订餐还不错好评
1,很好，送餐小哥服务不错，态度很好，准时，饭菜也不错，大冷天，赞一个
1,挺好吃的不错送的很快
1,美味，送餐人员态度很不错。
1,不错啊很好吃哦！送的好快态度很好
1,"又快又好,小哥态度很好,好评"
1,下雨天，小哥还提前送过来了，好辛苦，好评
1,"挺好的,服务态度也挺好的"
1,配送员人很好
1,不错，好吃，两个人没吃完，还会光顾！
1,小炒肉真心不是湖南小炒肉，坦白讲肉炒太老了，不推荐。鸭头不错
1,特别实在，实惠，量大，良心餐厅。味道也不错
1,很不错，菜品物美价廉～～～
1,空心菜份量少了点，汤不错
1,蛮好吃的就是饭太少了
1,今天送的真快，好评
1,送货时间挺准的就是对路不太熟悉～还可以吧～
1,最近迷恋金桔柠檬茶啊！
1,服务态度不错，骑手辛苦啦~
1,口味不错服务态度好感谢你们
1,今天真是超级快，感觉刚下单就送到啦哈哈哈
1,外送很快，给赞一个～
1,非常好。送的也特别快。棒棒哒！
1,好服务，就是习惯百度外卖的快速
1,口感好其他的不能比我喜欢
1,配送费越来越贵！！！！
1,态度非常好，速度快，味道好极了!
1,"好喝不贵,红红火火哈哈哈"
1,快递小哥辛苦
1,快递很及时，很方便很好
1,服务很好，下次还会订购
1,不错，就定他们家的1!！！！！！！！
1,送的及时，饮料很好喝哦
1,不错啊。快递很快，奶茶也还行。
1,很快。。布丁奶茶贼好喝
1,超好喝，快递小哥很热情，速度很快…
1,"挺好的,挺好喝的,不错"
1,好喝，也没有漏。，，，，
1,骑士很好，赞一个，很礼貌，辛苦了！
1,满意，速度也非常快。
1,味道很棒！辛苦送餐员啦！
1,经常定，特别好喝，快递给力。
1,大冷天辛苦快递大哥了。
1,超级快！奶茶一如既往的美味！
1,送餐很快，这么冷的天气辛苦了
1,外卖员名字跟我们家名字一样
1,服务态度很好饮料很棒！
1,在这样寒冷大风的天气里，等了一下午的奶茶终于到了，已经从热的变成凉的了。但仍感谢骑士的敬业精神！
1,"大风天,不容易,非常好"
1,"7点半下单,9点送到,今天天冷可以理解,外送态度也非常好,虽然等了很久,但是还是好评吧"
1,快递师傅特别好，大冷天真不容易
1,谢谢，非常感谢，那么等的天真是麻烦了，虽然等了会，但还是很感谢
1,"配送员辛苦啦,服务很好"
1,"喝了好几年快乐柠檬,第一次喝曲奇奶茶,个人觉得不好喝,本人还是更喜欢茶类冰饮,给配送员赞一个,大冬天的"
1,"一直都挺满意的~,骑士也很棒~"
1,"能送这个实在太好了,外送大哥辛苦了"
1,送餐很快，这么冷的天辛苦了
1,好喝，百度送的很快，非常感谢
1,一如既往的味道我喜欢的口味
1,"送货员态度很好啦！,非常感谢,辛苦了"
1,大哥很热情！不错!好评
1,大满贯奶茶！超大杯！
1,今天忙到忘了给我半糖，下次希望别忘。
1,快递态度很好，大冷天的辛苦了。
1,"速度很快,味道很好,很赞"
1,"骑士师傅人好,态度超级好"
1,服务态度好，速度快，值得表扬
1,师傅人很好，送的超级快
1,送的很快，师傅人也很亲和
1,好极了！送来还是热的！
1,非常满意！送餐很快！
1,十万个赞！最好喝没有之一！
1,百度骑士服务很好，全五分好评
1,"非常好,相当棒,很快，味道好好"
1,"一直很喜欢他家的东西,味好,便宜"
1,不错不错，快递师傅也给力
1,饮料好喝，送餐也神速。
1,岩盐芝士绿茶每单必点
1,一如既往的又快又好！每周都会在百度外卖买快乐柠檬家奶茶，送餐快，奶茶好喝，棒棒哒，开心～
1,快递员速度快服务好，辛苦了！
1,"元旦节晚上9点还在送真的让我感动,一直喜欢柠檬小子.蛋糕珍珠特别好喝,盐岩芝士加了珍珠很棒"
1,非常好非常快支持！！！
1,百度骑士特别给力，奶茶是经典大家都爱喝。
1,好喝好喝，超级喜欢这家的饮料，小哥送餐也很快
1,外卖小哥很赞！送的很快养乐多好喝！
1,再送小哥超帅！！！！
1,超级给力的骑士！饮料好喝！
1,"一直很好喝,这个很好喝"
1,好喝，送的很快，百度外卖我最喜欢用了！
1,每一样的很喜欢！会继续光顾哒
1,"非常好喝,一直喜欢,布丁好吃"
1,"送货态度非常好，提前了20多分钟，送得非常快，百度配送，靠谱！,奶茶一如既往的好喝，棒棒哒～"
1,嘿嘿把不错不错吧吧吧吧吧
1,第一次买冰沙，口感真不错！喜欢
1,师傅人特别好！送货超认真
1,外送小哥态度很好，很辛苦。
1,非常好！送货非常快！比美团和饿了吗好多了，以后就在这买了！
1,超快就送到了，喜欢百度外卖，真心赞！
1,"天气这么不好的情况下点外卖实在对不住外卖小哥,辛苦啦"
1,芋圆小丸子不甜～希望下次甜一点，真的没什么味道
1,虽然慢了些。但是能理解|
1,好好好！赞赞赞！态度好速度快！
1,非常快～辛苦了～很好喝
1,师傅人很好，服务很好，送得快~好评
1,"大雾霾的,外送大哥辛苦了！天冷注意身体！"
1,就是配送大哥忘记给我拿吸管了⋯⋯其他都很棒。
1,很速度，赞，奶茶也好喝。
1,师傅很好，服务很热情！
1,"派送员非常好。,速度快,态度棒棒的"
1,"建议再出款浓情可可，与岩岩的区别一下。,总体很满意啦！百度我支持你哟！雾霾天百度小哥辛苦你啦！百度公司一定要发给他们口罩哦，太不容易了！"
1,送餐员很热情，很好吃谢谢
1,小哥态度很好，两种饮料都很好喝
1,太赞了我家没电梯还能送这么快好感动
1,送餐员辛苦，奶茶好喝…
1,师傅服务很好.速度特别快.
1,百度骑士很快就送到啦！不错
1,热饮也有大杯就好了，中杯不够喝的
1,隔三差五儿的就点两杯味道不错！送餐员辛苦啦！
1,嗯，送货师傅依然很负责任呢！
1,速度快，服务好，必须赞一个
1,"还不错,送货很快味道也还好"
1,喜欢哦，每天都在喝，就是昨天忘了给我半糖了
1,百度骑士最牛x！！！！！
1,"非常好,非常快"
1,菠萝冻过甜，柠檬冰冻也不太好，但大满贯一如既往的好
1,送餐员辛苦！奶茶味道不错！
1,很喜欢喝他家的饮料！谢谢送餐小哥大冷天送来！
1,"挺好的,辛苦了"
1,超级好喝，几乎老点…送餐员师傅态度很好！赞！
1,简直神速…非常好喝。继续努力
1,配送非常迅速態度非常好给个赞
1,速度很快，奶茶也都封口了就不会撒出来
1,很喜欢，一周已经喝了四次了。最爱手榨柠檬绿
1,经常上班定这个喝，好喝，送餐员师傅，可好了。
1,快递员很客气辛苦，因为食物破损还跑了两趟，谢了！
1,非常好，现在每天都喝，下午一杯柠檬绿茶提神醒脑
1,喜欢。明天还会点。！！
1,好评啊，配送速度超快，这大雾霾天的辛苦啦~
1,辛苦送餐员，填错了地址又白走了一趟，辛苦了！服务态度很好！
1,换口味了尝尝、不知道是不是有进步
1,最好喝的奶茶，最给力的快递小哥。
1,超级好喝，快递小哥非常给力。
1,特别好喝，经常定，快递小哥非常好。
1,赞赞！加红豆味道更好！
1,好喝哦，但今晚我们作死点的无糖，感觉好酸，友情提示减肥的朋友点半糖
1,快递小哥特别好。奶茶棒棒哒！
1,很好喝，送餐也及时。。。。。
1,送餐员态度很好！辛苦啦！奶茶味道棒，谁让我是个奶茶控几乎天天喝…
1,这几天都在点，今儿给朋友点了几杯她们都赞不绝口！赞！
1,非常感谢！辛苦陈骑士啦！
1,好喝！味道超赞！送餐员也很准时！
1,"比上面提醒的时间早很多。,速度挺快的"
1,味道不错连着好几天了都在喝！
1,全都很棒，会继续支持。
1,"点了三次了,很好喝,送的也很快"
1,味道超级棒！送餐员大冷天的也很辛苦！超赞！
1,好喝！味道不错！送餐也很快！
1,非常喜欢味道不错，辛苦了！
1,这大雪天的，快递员给力！辛苦快递员了
1,还行，不错，快递也非常的快
1,好喝！快递小哥这么大的雨还送的很快！看他的湿哒哒的了！谢谢你们啦！！！
1,很不错，速度也很快。
1,师傅态度特好，真暖心，赞
1,大冷天的，快递小哥跟热情，准时，辛苦辛苦，很感谢
1,红豆布丁奶茶太甜。烤珍珠一直美味
1,"还挺快的～,味道也不错。赞"
1,都好喝。烤珍珠尤其好喝
1,美味，及时，经常订，希望优惠力度再大些
1,百度骑士特别准时，态度还特别好！
1,"快递员态度好,速度也很快"
1,味道很好，就是手滑点成冰的了，悲剧
1,非常不错，谢谢送餐的师傅
1,比预计时间送的还早十分钟，态度很好，点赞！
1,快递小伙儿特别热情.很好！
1,可惜不能更改杯型，又所有订单也不齐全
1,大冷天的，送餐员辛苦了
1,好喝，大雪天的非常感谢
1,。送餐员特好的说话。。。。。。
1,送货挺快的，送货挺快的
1,太快了，你是飞过来送餐的吧…
1,"送餐大姐人可好了，特别热情,五星！赞"
1,配送很快！送来还是烫的！很好
1,几乎每天都喝！…………
1,快递小哥态度特别好…辛苦啦谢谢
1,热饮有珍珠的。
1,百度的外送送了小礼品！很高兴！意外惊喜！
1,饮品味道一直很棒，有外送太完美啦～配送大哥服务很好，热心帮忙找零钱
1,服务很好！还送了吸吸冻！哈哈
1,好喝。。速度快。。。
1,非常方便！准时！很好！
1,奶茶好喝！送餐准时，到的时候还是热乎乎的！
1,奶茶超级好喝，喝了好多次了
1,他家的奶茶超级好喝。。。
1,奶茶超级好喝，推荐。
1,超级好喝的奶茶。。。
1,送的超快！！
1,速度很快，32个赞，服务很好
1,非常好送的很快，，，还是热的。
1,准时到达，太给力了！
1,速度很快。而且我找不到钥匙也耐心等。还安慰我说不着急。人很好。东西很好喝。下次还会买
1,特别感谢快递师傅！这么大风天气还准时送到！辛苦辛苦…谢谢啦！
1,速度很快，唯一不足就是第一次下单莫名的失败了。。。
1,很好喝！天天都喝～～
1,配送不错、态度也不错
1,"师傅人很好,，送的速度也很快。"
1,很棒的骑士！！！！！
1,还不错，绿茶还是个人喝不惯。。。奶茶给赞
1,小哥长得帅态度还好，好评
1,很快，没有撒，必须赞
1,很快，也很好喝，不错
1,很好送货大哥
1,奶茶很好喝，快递小哥也超给力！赞一个，辛苦了。
1,服务热情！点单时忘了一杯，给快递小哥打电话，有帮忙捎了一杯！
1,"有效率,非常好"
1,热情！速度快！
1,超快！服务好热情
1,"超级快,送餐员超级好"
1,很快好开心
1,优惠活动不错
1,非常方便！很好！
1,"杯子坏了,不过饮料没出来,其他都可以"
1,赶上雨最大的时候，外卖小哥很不容易
1,非常非常好！服务超级赞！
1,非常感谢百度外卖的服务，大哥因为找不到地方饶了大半个圈。
1,速度快態度好
1,❀⚘送༵来༵不༵是༵很༵凉༵了༵但༵味༵道༵还༵是༵不༵错༵的༵⚘❀
1,"还是那个味道,都很喜欢"
1,几乎天天都喝
1,弹珠和芋圆小丸子很好吃
1,"每次都买珍珠奶茶,很好喝,送餐速度也快！推荐购买"
1,送的超级快，送餐员很辛苦。快乐柠檬一直喜欢喝
1,奶茶很好喝，快递员也特别好
1,很好，很喜欢，希望出更多新口味。
1,送餐师傅很好呦！饮料好大杯！吓死宝宝了！
1,"灰常好～,经常点～"
1,"师傅相当神速啊,好评！"
1,外送小伙儿很逗
1,"味道超赞,送的也快提前了半个小时"
1,足不出户享受清凉，快递小哥谢谢啦～
1,速度快服务态度好
1,送货很快哦
1,叫了好几次，真心好喝
1,速度很快哟，女儿的岩盐芝士可可有点咸哈哈，我的的大满贯还是比较赞滴！
1,非常好，快递非常不错
1,每天都会买上几杯，快递非常棒
1,好喝，速度也很快，值得赞
1,"味道很不错,送餐人员态度也很好,很赞,以后还会来"
1,一如既往的好
1,少年很可爱—v—
1,非常的快、有温馨提示！
1,下次记得带够零钱
1,奶茶很好喝
1,送来的挺快的～味道不错
1,送来的非常快，饮料都非常的不错呦，超级喜欢
1,"味道很浓,而且还可以换不喜欢的布丁,很灵活的店家,赞一个"
1,感觉没有去店里的好喝
1,速度超级快呀，很好喝，冰爽
1,好的还可以
1,很棒！太快了
1,送餐快，非常好
1,曲奇很好喝
1,超级超级好
1,很准时！口味很好，大爱可可小丸子
1,"速度很快,谢谢送餐员,辛苦"
1,挺好，没撒！
1,岩盐芝士的也好喝
1,好喝～不用去外面买了～
1,下雨了，感谢送餐员
1,"非常好,辛苦送餐员"
1,挺快的，好
1,喜欢多年。非常好。
1,谢谢外送人员。在雨天的家里还能喝到快乐柠檬
1,"东西很好吃,送餐速度和态度也很棒"
1,本次订餐很愉快！
1,挺好的！下次还定！辛苦送餐小哥
1,"米饭好少啊！对我这女生来说也不太够哪！,水煮牛肉还是很好吃哦！"
1,骑士小哥辛苦啦！非常快
1,菜很好吃，就是炒饭这次给的有点少。
1,快递小哥服务特别好，餐送到还热乎乎的，正好吃。菜品不错。
1,"饭很少,要知道就多定两碗了,送餐师傅很棒"
1,"土豆炖得特别好,适合老人吃,谢谢"
1,速度比预计快很多，菜品不错
1,"辣子鸡越来越少了。,！！！！！"
1,送餐小哥真快，菜也很好吃
1,好吃，好吃，江山城的烤鸭真是挺好吃的
1,挺好的，挺不错的，味道不错
1,菜有点辣，菜量挺好，量大。味道挺好。
1,态度很好，效率很高，给十分赞
1,每次量都很足哈！超级赞，好吃
1,送餐很快，可是为啥没餐具呢，用手吃？幸好送餐师傅给了两双百度餐具。
1,百度骑士速度好快啊！而且服务态度超级好～～江山城的饭一直很好吃，走了外卖很方便呢！
1,麻辣香锅做的真好吃，
1,很棒。。很快。。很好吃
1,他家菜很好吃，有求必应，送餐速度很快，赞
1,速递小哥辛苦了，饭菜到手还是烫烫的，在这寒冷的日子，谢谢啦！
1,很好吃，菜品味道很好，有要求都很爽快满足，赞
1,点了三个菜，师傅拿来六盒子，量很大的。千叶豆腐和包菜咸了点，鱼香肉丝咸淡刚刚好，口味很好，干净卫生，没想到这么好，还送了几颗山楂，希望品质会一直保持，还会再来滴。
1,挺快的味道不错，推荐大家选择。
1,服务热情，送餐高效，很满意
1,味道不错，挺讲究的，私房豆腐很好吃
1,有一点问题，就是餐盒用的太多。没有必要
1,在家附近吃了很多年，烤鸭一级棒。
1,送餐快，菜品还是热的.服务也好
1,很不错，以前都在店里吃，现在方便了
1,不错不错，味道可以！速度也还可以
1,很棒的哦，第二次订餐了
1,丸子很嫩，拔丝山药还给配了一袋水，呵呵，好贴心
1,很快很好吃更给力会继续支持的
1,不错哦，感谢百度骑士！
1,这家店和百度外卖太配了，炒菜速度也快，送餐速度也快，全家都很满意，点了四个菜，宫保鸡丁、鱼香肉丝、酱茄子、酸辣土豆丝，土豆丝是面的，也挺好吃，其他几个菜品味道都不错，多吃了碗米饭，作为送餐能这样的效果超级知足了！还会再来！
1,速度很快，菜的味道也好！不错！
1,毛血旺辣度正合适，料很足，比眉州的性价比更高
1,宝宝很爱吃荷兰豆，甜甜的
1,"第二次买了,挺好的,以后继续"
1,"很好吃,送餐速度快,下次再来"
1,送外卖小伙服务真棒，很尽责！
1,挺好，继续保持！
1,味道很棒啊
1,配送师傅速度很快，态度也好。
1,"百度外卖超快,有些菜没了，换的"
1,"百度外卖，赞。,菜不错就是请注意包装。"
1,"百度外卖，赞。,店家的包装要提高，都洒了！！！"
1,店家给力，骑士哥们也给力，辛苦了谢谢，，下次还点。
1,外卖小哥很可爱。饭少。zz
1,好吃，服务也好，满足了我的需求
1,配送速度很快，服务态度很好
1,非常好的服务，跟鞋你们
1,谢谢骑士，辛苦了～～
1,不错，方便，挺好吃的
1,那个汤的味道是真心的接受不了，其他的还好，味道比较不错。
1,第一次吃鸡米饭超级好吃！送餐也超快！
1,"排骨鸡里没有青笋么,下次放点吧"
1,"汤很好的说,送餐很快,棒棒da"
1,"吃的挺好的,以后还会点别的"
1,帮女朋友订的，味道很好。
1,鸡汤咸死，无法入囗！
1,不很好吃，微辣就是一点都不辣
1,撒得一塌糊涂，味道不错。
1,不错，味道挺好。值得点赞。
1,少一盒米饭，只送了三盒米饭，我要了四盒
1,快递给力，排骨饭和鸡饭全是肉，味道好，汤都拌饭了，一个女生吃不下一份。
1,很香，也很好吃～只不过排骨里面的碎骨头有点多。
1,还要说十个字！？好好好好好
1,微辣一点都不辣啊
1,味道很好，服务很快，值得拥有
1,喜欢吃黄焖鸡，味道棒棒哒
1,味道不错，送餐速度快！很不错大赞
1,嗯，好吃，黄焖鸡真是吃一家一个味，我喜欢这个味道，棒棒哒
1,卡超级快就送到了谢谢谢谢
1,满意，很好，配送给力食物好吃
1,味道很好，可惜吉野家的量一直不太多
1,很快，很好，经常订，谢谢师傅
1,真不错，价格便宜实惠，分量足，最满意的一家
1,物美价廉，很好，谢谢。
1,这大冷天的辛苦骑士了！谢谢您！饭很好吃！
1,味道一般，但是服务和饭菜质量很好！
1,到的时候饭饭热乎的呢！又快又好，大冷天的真辛苦，感谢！
1,米饭有点硬，有点凉了。
1,百度骑士很棒，又快又好
1,特别好吃还会来!!!!!
1,东西好，对川人不合口味
1,很好吃。也很速度。服务态度也很好
1,"送餐简直是坐火箭来的,太快啦。送餐小哥很热情。非常满意"
1,送餐非常速度，负责人，热情！
1,"不错,第二次购买了,很喜欢"
1,服务和味道都是不错的
1,看着不错，饭量也还可以。
1,"飞速送达,谢谢配送员,菜品也不错"
1,确实很快，食物也不错，比吉野家便宜。好好好。
1,服务周到。很贴心。我喜欢。
1,送餐速度很快，非常好
1,外卖小哥送的很及时，服务态度很好。给个赞！
1,百度骑士很快，菜品也不错，应该是有所升级
1,百度骑士非常快，菜品也很不错，好像有所升级
1,盐酥鸡太不正宗，不好吃，饭不错啊
1,配送很快，师傅态度很赞。非常感谢百度外卖！
1,"骑士人很好细心超快速赞一个,饭还好量大"
1,"太实惠了,味道好,好好好好"
1,"快递小哥很好,赞一下"
1,"挺好味道比较淡,还好给了盒榨菜,还不错"
1,很不错，饭菜量都很足
1,速度真快，20分钟就到了，而且味道不错呢
1,"速度很快,赞一个快的小哥"
1,很好！外卖小哥很给力！！！
1,送餐员辛苦，饭也好吃，继续购买
1,快递员辛苦了，菜品也不错
1,"都是肉,有些干,肉真的很多,多一些汤汁就好了,味道还不错,还送了个速溶咖啡"
1,速度很快，用了不到20分钟就到了，东西也很不错，赞一个！
1,送餐快，味道可以。如果有满减就更好了。
1,速度很快，东西也不错，赞一下
1,好评一个。干嘛非要10字以上！！！
1,送餐的哥哥好快呀，态度超好，这大冷天的真是辛苦了
1,味道很好，米饭软硬也刚刚好。
1,快递师傅很好，饭也不错，味道不错，尤其量也足，那米饭给的，都吃不了，太少见了
1,很好吃，量大。快递小哥，服务一流。
1,很好！份量很足，香干回锅肉里的萝卜干很好吃，每份饭还送了咖啡，谢谢！
1,快！外卖小哥特别棒！
1,饭到的很快，服务非常好。
1,"很好,外卖小哥服务也很好"
1,"特别快,感觉定完没几分钟就来了,送餐小哥很细心,赞一个"
1,送餐小哥很不错，很有礼貌啊！
1,"很好吃量也很大都吃不完,赞一个"
1,这饭太给力，吃不完都
1,挺好，速度快
1,配送很快，赞！套餐份量很足，味道很好，特价价格也很便宜，非常划算！
1,好吃，好吃，好吃，重要的事情说三遍。
1,很好吃，满减时买价格也合适~~
1,挺好的，送的也挺快的，米饭再好点就更好了，感谢送的coffee
1,送餐员很给力！大大的赞
1,"速度挺快的,赞一下"
1,送餐员态度很好，产品质量也好，满意。
1,不错的一家小店噢，以后会经常光顾的，推荐黄焖鸡～
1,"味道很好,只是最近双拼换了餐盒,参合不大好,送餐及时,送餐小哥认真负责,态度很好"
1,送餐小哥服务太周到了
1,"非常好吃,量也大,送餐小哥工作认真负责,棒棒哒~"
1,吃起来就知道很干净，入口很干净的外卖不错
1,送餐的小哥非常赞，大冷天的，真辛苦！态度速度都超赞，必须五星！
0,此次肉相对以前很少
0,送餐服务有待提高！！！味道不用说！晚了半小时！
0,"味道还可以,送餐速度超级快"
0,太差了，送来的饭菜汤都漏出来了，还不承认，太差劲了，饭放这就走了，太垃圾了
0,太次了，一个半小时了才送到，面都驼一起了，没法吃扔了，以后再也不定了
0,怎么个意思，送外卖的还管我几点订餐么，你可以不接单，还管到我了，什么态度
0,"分开了好难搞，本来就图个方便,搞这么麻烦"
0,新品，还贵，不是一般的难吃，全部扔掉。
0,等了好久好久。。。。
0,西红柿汤还行，就是感觉米线不是太精到！米线有待提升.
0,宫保鸡丁吃着一般，不太好吃，下次换别的口味！
0,送餐太慢，打电话摧态度比我硬
0,接电话那女的态度差的像借她家米还她家糠似的，女鬼似的凄厉，打两次电话两次如此，一直去他家吃饭，订餐，发誓没有第二次
0,"我真的是特么的服了这个送餐速度了,2个小时,我特么,催了1个多小时,东西还真挺特么好吃的,就是太特么慢了,以后大伙们想吃就去店里吧,定了肯定耽误事儿,今天我特么就认了"
0,"送餐速度一般一小时左右,等待太漫长啊,味道还不错"
0,送了快两小时才到！！
0,1、太辣。2、有花椒，不喜欢吃。3、粉条用筷子夹容易碎，用勺子又太长了。4、如果送一包醋，味道会好一些。
0,"21的至尊肥牛套餐,居然连饮料都没有,着实被图片坑伤"
0,"11:30,的餐，下午三点才送到.,还是打百度投诉的结婚！"
0,"我真服了,呵"
0,送的太慢了，送餐员连路都不认识。
0,"漏给了,2个半小时之后才拿到外卖,百度外卖真心渣"
0,汤汁全洒了
0,等了快两个小时，电话打不通，催单不管用，米线没豆花没配菜
0,番茄贡丸豆花米线以后请叫番茄贡丸米线，好吧？！想给零分差评！！！
0,番茄贡丸豆花米线以后请叫豆花米线，好吧？！想给零分差评！！！
0,豆花米线不给豆花！！！！！
0,饭菜不错，量足，但手机没信号就没接到电话，打电话过去服务员态度特别不好
0,神速啊。特么完了快一个小时，真牛逼
0,比预定时间晚到了一小时又15分钟，送餐员说找不到地方，拿回去换人再送，不会打电话问吗？？？？？我们这里每天送很多餐，就你找不到，猪脑袋！以后再也不定了
0,没什么味道！
0,"换过两次套餐,都是17换16,前后2次,2块钱,都没退"
0,说了要醋要蒜一样没给送，要两个卤蛋给了两根肠，也不打电话说一声，太不上心了吧？！
0,太慢了快要饿傻了。。
0,"整体还不错,就是太肥了,而且都凉了,送餐态度不好"
0,"面成了面疙瘩,全倒了"
0,味道也就那么回事吧，米线给的量好大，一半都没吃完
0,差差差，不到十一点定的，十二点半才到，整整脱了一个多小时。打电话说到楼下了，三十分钟也没上来，爬楼梯也十几分钟到了吧，
0,就那么几步路远等了一个小时！！菜还咸的要命真是醉了，再也不想来了
0,太慢了啊、
0,"送个北科大，500米,送了半年,够牛逼的"
0,我点的渝川番茄米线，口味一般，没有期望那么好。为了错开用餐高峰期，11点01分下的单，结果一个小时后没有到。一直催了N次单，店员总说送出来了。从小渝府到北科一共5分的路程，加上做的时间，送其他人餐的时间，怎么算也用不了三个半小时。不得不想点了餐，是不是去种菜种大米了！好好的生意也是会做黄的！经过几次交涉，餐送到。口味还一般，真不知道为什么小渝府有名的米线是怎么来的？刷的吗？这个样子，也就吃这一次了。
0,"11:46下的单，首次送达就是13:34，而且酱香排骨豆花米线仅有菜没有米线。,再次送达米线时已经14:10。,等个饭等了2个多小时。"
0,太慢，等了一个小时
0,"速度达不到40分钟,希望如实描述,。米线油多,倒进全部米线后会溢出饭盒,很不方便,希望换更大更高的饭盒"
0,面油有点大，汤洒了，送餐态度不咋滴
0,服务态度实在是受不起，十点二十点的餐，一点整送到，点的河粉都成一坨了！而且第一次打电话给商家，世纪就隔了一条马路的距离，店家居然告诉说你们那么远？！然后名正言顺的在点完餐2个半小时才到。更搞笑的是后来多次打电话居然根本全部打不通，你tn是屏蔽我了是吗！由于送餐的绝对及时，是在临上课前两分钟到的，于是，我们很愉快的中午定了份晚上的外卖！！要不是评等级最低给一星，我真是一个星都不想给！中午简直是气饱的。
0,速度很慢，我今天打电话催的时候，那边态度很差，我说那明天不点了。她说“可以”，电话撂了。
0,我点了一个蛋一根肠一份套餐！给我送来两份腌菜没有蛋！！
0,"送来时都坨了,没一点汤了"
0,"送饭的人都是脑残吗,能不能找个智商高点的来"
0,等了98分钟，面陀了。
0,没有也不告诉一声
0,垃圾，慢死了
0,送餐及时，菜量不小，足够饱。
0,菜都撒了，米都让菜汤泡了，比预计晚了20分钟，这次特别不满意
0,蒸蛋分量大
0,梅菜好多，海带味道怪怪的
0,"我订了广味烤肠,但没有给我送"
0,味道跟店里吃的完全不是一个样啊，送餐太慢，汤也凉了
0,我这是第一次订餐，也是最后一次了，送餐用了一个半小时
0,味道还好，送餐速度满了点，应该增加配送人员
0,"接电话服务员态度很恶劣,语气很差,说我们太远,下次不要在订餐时间叫餐,都在别人不吃饭的时候点餐,在她家点餐的朋友慎重,一般两个半小时送达"
0,史上最慢最差的外卖。等了两个小时，不能再差
0,科大这两步路送了一个多小时，啥也不说了，再也不见
0,实在太慢，送到的时候凉了，汤还洒出来了，说是量太多。。。。太多。。。。
0,"太慢了,十一点定的"
0,咸菜炒肉，很一般
0,订了两单，还分别送到了两个地方
0,"商家态度差,，送来的餐品全洒了"
0,商家态度差！说了备注忌口，也没按要求。
0,比约定时间晚了整整40分钟！问送餐员怎么晚那么久，居然说饭你要不要吧？！真么见过这样的，还没一点歉意。百度客服也就那么回事，形同虚设！
0,送来的太早，都不知道备注送达时间有什么意义！
0,店家真是操蛋，竟然给少送了一份，另一份12：20送到。
0,鸡块里一半是鸡脖子……
0,"送餐员,态度较差"
0,昨天忘了送鸡蛋，今天提醒了还是忘了
0,土豆有些硬，或许土豆比较陈
0,我的两颗卤蛋呢！！
0,少给了我一盒米饭
0,"菜里有花椒,，而且又吃到,塑料线了。。。。。。。。"
0,和在店里吃差距好多，米线都是碎的一段一段。还少，感觉是把剩下的碎料给我了，酸菜肉丝配菜更是没有看到，还得拎着饭盒去店里要
0,送餐速度真慢，送来的都是凉的，让人怎么吃
0,希望可以增加分量，明显分量少了许多，不要走下坡路，我是小俞府的忠实粉丝
0,没有年前分量多，在走下坡路
0,没有年前分量多了
0,可惜不能发照片，不然我一定会让别人看看筷子和纸在菜里的照片
0,店家根本不看备注内容，让给开发票也没给开！！！！
0,肥牛有点硬
0,我一度怀疑自己订的是拌面
0,送来的太晚，而且备注了不要辣，结果超辣
0,送来的太晚
0,要么就别搞减价，减价就给我减东西，汤都喝完了都没有看见一块肉，也是醉了
0,"说实话,不太喜欢口味,米线太多,汤料有点酸菜桶面的味道,反正不太喜欢"
0,少了两个蛋
0,以前的优惠已不再
0,少送了一瓶加多宝
0,"店里马上放假人手不足，等了一个半小时，终于送来了。,不过菜做的都比较咸，原来配的圆白菜没有了……换成了类似腐竹细丝但不能确定是什么的东西……还不如圆白菜爽口,红烧鱼块淀粉较多"
0,迟到了半小时
0,你们就是这样的吗。鸡汤是酸的，放了几天才给我们喝呀。妄我这么支持你家
0,订歺后2小时才到
0,鸡汤很是失望掺了一半的水。订歺后2个小时以后才到，让我的面泡糊了，吃了想吐。
0,第一次吃到这么多姜的鱼香肉丝，不好吃，另外送达延时
0,提前2小时定，迟了一个小时才送来
0,服务员态度死差，建议改革
0,饭还凑合，一直都订你家的，今天居然晚了1小时，打电话询问服务员还一副我欠她钱的样子，真是够了，哦，呵呵。地址离店走路也就20分钟，现在居然完1小时
0,晚到一小时
0,一个半小时才送到，催单态度还不好！
0,过节送餐人手紧，可以理解
0,"凉菜的盒子竖着放,凉菜都洒到包装袋里面了"
0,菜超级少！！
0,菜里全是汤洒了一袋子。。
0,饭不错，速度慢。
0,饭菜做的不错，送餐速度有些慢，要是能快些就好了
0,从下单，到配送，前后两个小时，多次打电话催，等了一个小时先是说送餐的人车坏了，回去换车，过了半个小时再催，又说换了个人送餐。又等半个小时，还没有送到，饿得不行了，就自己出去吃饭。两个小时才收到送餐员电话，结果告诉我他的地方还没对。本来是加班，想说订餐可以节约时间，结果可好，气死了，再说，饭菜做好了，即使两个小时送到了，还能吃吗？
0,时间太久，等了2小时了
0,鱼块味道一般，小咸菜味道不错，米饭有点多
0,河粉量很大，配汤略咸
0,"汤倒了,味道不错"
0,10:23下的单，提示卖家已经在准备了，11点左右打电话过去问，店家说正在准备打包，11点才送外卖，我说好吧，快点。11:26我取消订单。（我已经等了一个多小时了）11.34接到电话说，餐到了，我说都一个多小时了，我不要了，店家态度非常恶略，说胡说八道，哪有一小时！我说你回去看我的下单时间。对方就直接挂电话了。
0,口味还行，油较大
0,"特别慢,建议大家早上订晚上的饭,这样大概能准时送达,服务态度极差"
0,"慢的可以,服务态度极差"
0,"10点订的外卖,12点40送到,呵呵呵,这种外卖店还好意思送外卖我也是笑了,11点40打电话告诉我早出发了,12点10分打电话告诉我等等,12点45送到了你觉得我还要么？,电话服务是这样的,她说“你还要外卖么？不要我就挂电话了”,就这种服务态度,真不知道是怎么好意思做服务业的"
0,"不到500米距离,不算预定,送了两个多小时"
0,速度太慢，超出预定时间2小时才送到
0,今天这顿午餐吃得着实不爽，10点多订餐，要求12点到，结果迟到了40多分钟，1点就上班，还有时间吃饭吗？配送员竟然和我抱怨说这边太远了，这个理由太扯了吧。好吧，就不说菜汤都流的到处都是了，着急赶快吃完上班，一口菜进嘴里，一个比鱼刺还大还坚硬的塑料碎片卡在嗓子眼，估计是你们塑料勺碎了吧？怎么还掺在菜里面？幸好我及时取出，现在嗓子眼还疼着，物证还保留着。这种卫生，着实醉了。
0,压根儿没送到！！！
0,味道还可以，就是饭菜全凉了。。。本来11点前点的餐，硬是快到1点钟才送过来，我真是服了这送餐速度了。。简直刷新了我对于外卖服务印象的下限。
0,太慢了，以后不点这家了！都凉了
0,实惠、好吃！好慢啊……
0,东西差难吃，吃一半就给扔了，另送餐人员素质也不高，
0,没以前好吃了，还跟乌龟一样慢死了
0,少给我个爽口菜。打电话一直占线。今天给你扣5元，爽口菜钱
0,真垃圾，如果有负数就好了
0,米线都碎了！一股搜了的味道！！！怎么吃？
0,打电话给商家2次，太慢了。河粉夹不起来了，坨了，一碰就碎。味道也不好。再也不会订了
0,爽口菜吃出了烟头
0,量挺多的，味道很不怎么样！
0,"早送了半个小时，吃的时候都凉了,还少一根烤肠，强烈要求退款"
0,最难吃的一次，菜明显是剩下重做的，吃两口就扔了
0,点了两瓶啤酒就送了一瓶
0,饭还行，就是送的太慢
0,牛肉嚼不烂，肉比较死，豆芽水煮一样，说实话真的很一般！说的事实！
0,不会再点第三次了！！！
0,"晚了45分钟，套餐有半个卤蛋，但是没有饮料；,肥牛饭味道比较一般，但量很足（ps,配菜是豆芽菜，个人最不喜欢豆芽菜,赶上了……）"
0,吃不饱，量太少
0,1送餐时间非常不准，不管是提前送还是迟送，2西红柿鸡蛋太咸，打死卖盐的了，3买炸酱面也不送汤了
0,慢死了，气死了
0,等了五十分钟，受不了的龟速
0,送餐太慢太慢了，送来粉都坨了，备注写了不放葱，最后还是放了！
0,我的烤肠怎么没了！！！！！
0,不能更慢了
0,送的很慢，但是味道不错，可以再继续尝试，只是希望提高送餐速度
0,点这家三次了，每次都把东西洒出来，让人怎么吃呢！
0,都没送怎么就叫订单完成了？
0,送的好慢两点多给送到的，，什么事啊
0,等了几乎两个小时，和预估40分钟未免相差甚远？
0,"你确定11点10分是你送达,的时间？到12点半才来，菜还撒了，很无语。。。。"
0,除了米饭不好吃其他都还行
0,不好以后不回来了
0,热菜太少，这是冬天季节
0,菜不错，只是送餐服务不好，时间总是不准，公司离订餐的地方300米，送餐时间误差45分钟左右，我订12点送，有时11点10送来，有时12点45送过来，你提前送来面都坨了，怎么吃，什么玩意。
0,过了一个钟头，还是没送到
0,能不能到了再打电话，害的我非得在楼下等半天
0,宫保鸡丁只剩黄瓜了，比以前垃圾多了，不再爱了
0,"不错,量多。能吃饱"
0,平时每天迟到30分钟送达，今天选择11.:45，11点就送过来，是不是故意的。
0,这个时候我等得睡着了，做了个梦：餐到了
0,口味还行吧。。。就是送餐太慢太慢太慢了。。2个小时啊。。
0,"淡,只有咸味,前后差一两分钟定餐缺隔了半小时送了两次,还少带过来两根火腿,不过又给顺路退了一次钱还好"
0,经常吃，味觉无法识别了！
0,小渝府，不再爱了！
0,10点20下单，1点半还没有送到，只能取消订单。
0,实在是一般啦
0,送的太慢了，菜都凉了。
0,真心垃圾，今天中午又在这叫个外卖，等了一个多小时也没来，打电话想取消还不接，催单都催好几遍，一个多小时了你们是去种菜了吗？
0,太慢了，路程距离应该只需要5分钟就能到，但是用了这么久，饭菜都凉了
0,坦白说，饭菜挺好吃的。但是送餐太慢了，快2个小时才送到，送餐员也不容易。希望能多些送餐员配送。因为送餐时间太长非常影响用户体验！
0,乌鸡汤一块鸡肉都木有！！！！！
0,第一次见服务态度这么差的，点了一份套餐和一根烤肠，烤肠居然直接就没有送过来，送到的时候饭菜都凉了，也是我吃过的最难吃的梅菜扣肉，再也不订他家的了！！！
0,我点的是套餐，是不是应该有饮料？反正我没有。送餐时间太长了，差不多了要40多分钟
0,"我马上就要饿死了,跟我说马上就到,马上就到,这马是没吃草吗,下次别放那咸菜了,味道难闻的要死"
0,3份饭，一双筷子都没有。
0,太慢了、恶死了
0,慢慢慢慢慢，岂是一个慢能够形容的
0,别的暂不说，打了四五个电话，9点多定的餐2点了早已经上班了才到手里，你可以早点说送不到啊，送到当时真想不要这份饭，都不容易，想想没必要而已，欠一个解释，我想这也是最后一次订了。希望你们也要注意，类似的错误不容发生。
0,明明走路十分钟就可以到的地方，可是十点半之前订的餐，十二点半才送到，米线全都烂了，真是没法吃
0,"灌汤酥肉套餐,也许是我的味蕾坏了"
0,太差劲了，13:30才送到，晚了2个小时。谁这个点吃午饭啊？直接让他走人，以后再也不定这家了。
0,10点半下的11点半的单子，13点05才送到。。。
0,太慢了，等了两个多小时
0,辣死了，吃完整个人都不好了。。。
0,饭价随百度减免搜搜的涨………………
0,一个半小时才送到，图片上的饮料也没有，一点也不值21元。以后没有优惠卷活动了肯定不吃了。
0,送餐不带零钱还要自己出去换也是醉了
0,为什么没有饮料！！！！打电话说外卖就没有饮料，那你图片是干嘛用的呢。怎么不写里边说没有，这算欺骗吗?
0,外卖态度比较差。
0,叫了好多次你家的，这次送餐的态度特别差
0,比预期送餐时间晚一个多小时
0,不要了还显示送达？
0,"什么破东西啊，10点半定的,1点送到的"
0,菜品偏咸，味道也一般。打开包装里面菜汤全洒了。
0,"换成了酥肉套餐,不好吃。。都是肥肉"
0,广味烤肠漏送了！
0,菜里吃到塑料。。。。。。。这是什么情况！！！！
0,口味一般，送的太慢
0,送餐速度还行。味道惨不忍睹，就怕菜的味道太淡，特意点了宫保鸡丁，这居然都能做的那么淡，不咸不辣有点腥。以后估计不会点小渝府的外卖了。
0,此次送餐时间还算迅捷，可惜原来的20-13活动咋不继续了呢？
0,根本没有送
0,配菜变成包菜，差评！！
0,还行，配菜改成包菜了，差评！！！
0,这个饭几乎没有肉，竟然连土豆都很少，根本就是给人吃白米饭，还卖这么贵，根本不能忍。
0,非常难吃，鉴定完毕，不再从此家订餐了、
0,"都没有送到,还让我评价18:40订的,20点都没有看到饭~~~"
0,无论是外卖还是实体店，一直在支持小渝府，但又被无故取消订单了，还因此错过订餐的合适时间。作为老顾客很不开心。
0,"没送烤肠和卤蛋,餐厅电话打不通"
0,"口味不错，就是,时间有点久了，粉有点碎了，还有原来是一个卤蛋啊，我以为是两个！"
0,订了份蕨根粉和鸡蛋炒饼，蕨根粉特硬，中间还有硬结，就把上面黄瓜吃了，炒饼特多油还没味，后来没办法，吃了几口送的萝卜咸菜硬咽下去几口炒饼！
0,送餐太慢了，又是90分钟
0,除了慢，其他还错
0,"我住在北京科技大学里面,不到500米距离送了1个半小时。菜基本凉了,而且洒了一半,弄得我手上,钱包上,桌子上全是油。做的再好吃也是差评"
0,等了两个小时。。。。
0,说了希望多送点泡菜，结果连一点都没看到，
0,"还行,就是一个多小时才送达"
0,此单根本没有送达，打电话过去说没有做，都快两小时了，你告诉我没有做？垃圾垃圾。百度竟然还确认送到了，岂止是垃圾，简直就是垃圾
0,垃圾中的战斗机，几天之内定了三次餐，两次都超过一个半小时，打电话好多次才联系上，然后说订的餐没做或者找不到了。什么垃圾，接不了业务就别做这活，动不动让老子快两点才能吃上中午饭，坑爹呢。
0,完全凉了，而且我不知道送外卖你2块钱零钱都不准备吗？上次是100找不开，让我去楼上重新拿，呵呵，这次TM2快也找不开，等就等，饭菜全是凉的
0,面送来都成一团了。。。
0,比较多，女孩子吃不完
0,味道可以。
0,红烧排骨，来的全是笋子。店里面卖的怎么和送的差距这么大？弄不好外卖就别弄，又慢又坑、
0,可惜一直给的五星好评了。送外卖的伙计居然说以后不给我送了。理由是我们楼只有我一个叫餐的
0,连筷子都没放，菜很少
0,不知道是不是忘了放盐，肉很腻，味道不太好，小菜还是不错的
0,挺快的，但是味道不太好
0,"太慢了,花儿都谢了"
0,太慢。菜全洒了。
0,一般吧，套餐里的菜有点凉了
0,"太慢了，,总共300米的距离，,11点20订的，1点多才打电话告诉我到了。"
0,"挺搞笑啊,等了一个半钟头终于打电话通知我去取餐,然后送餐的竟然说等了我半个钟头了,下次这样我就拒收了"
0,我去，怎么写了个送达？？？根本
0,送的太慢了，真是气死人
0,很难吃，送的也慢，整整迟到了两个小时
0,菜不符！而且难吃
0,送餐速度太差，12点下的单子，下午2点还没有起送！还得我打电话催才给查询是否送餐了。害的我没吃上饭
0,送餐员态度很好，唯一不足是送餐时间较长
0,味道一般，面只有辣、麻，土豆丝没味道
0,夫妻肺片很腥，吃过的最难吃的夫妻肺片
0,强烈建议别买
0,菜太少了。。服务员服务超差
0,送餐时间平均为一个半小时，等两个小时纯属正常，各位可以试试
0,送了一个多小时，都凉了，不好吃
0,去你妈的，什么几把玩意，气死我了
0,好难吃，说是酥肉，其实都是大肥肉，味道也不行，第一次百度订餐，印象很差劲
0,"唉,送过来一份饭,都没地方放,里面的汤全撒出来了,直接扔了"
0,送餐速度很慢，菜量小，菜品也太差，根本对不起这个价钱。
0,送的不准时
0,送餐不准时
0,"我要的是酥肉套餐,，结果送的是扣肉。这怎么评价呢，扣肉味道还行吧。"
0,速度慢，第一次订餐就迟到，晕
0,送餐速度太慢，打电话想问问做了没，服务员的态度太差，竟然就喷顾客。所以以后都不会再订这家的
0,s送餐太慢
0,快点吧！饿了
0,太慢了，不过挺好吃，还有就是我拿错别人的了，亏了我5毛钱
0,同宿舍三个人一起点结果分三趟才送过来，等了两小时才送到，从来没见过那么扯淡的！
0,我的卤蛋没给。已经不是第一次少给东西了
0,比别人先定还比别人后到是什么原因？
0,比别人先定的还比别人后到是什么原因
0,"太慢了,慢的不行,而且外卖肉少的可怜"
0,差！！迟到一个多小时！！！！！！无力吐槽，减13了服务就跟不上了，搞活动干嘛？！！
0,晚了半个多小时，菜汤都撒了。。送餐员没到就给打电话说到了，结果又在楼下等送餐员。。。。服务太差！！！！差差差差差差差！！！！百度快变回减8块的活动吧，别减13了，根本就不保证服务质量
0,为什么里面的饮料没有图片上有，送达没有，而且还贵了2块钱！
0,凉菜量很小，味道不错，感觉不怎么实惠。
0,必须差评了，配送的不知道怎么搞得，餐盒都放不好，牛肚里的红油流的满塑料袋都是，看着这恶心！烤肠被压在河粉下面，让红油给串喂了，太次了。
0,下单确认后一刻钟才被告知没法做，我也没辙了…
0,饭还可以，速度太慢了，5分钟的路程，足足等了1个多小时
0,送餐没有原来快了
0,来的太慢！！！
0,太坑爹了。订完了一直是确认单。但是一直不给我送。二点了我同事去店里给我取说是卖没了。卖没了为什么不及时联系我。而且我的状态也还是确认状态。小俞府你已经坑爹两次了
0,"唉,太慢了"
0,太少，太难吃！！！
0,1个半小时才到
0,未按照约定时间送达
0,太慢！三个多小时
0,生生把中午饭送到了下午，将近2个小时，而且不止一两次，每次给机会，每次都如此，走路过去不到十分钟的事！等电梯的2分钟都要打电话催，一碗汤不如叫一口汤好了。差评，以后再也不订了。
0,如果你刚吃完，点这家没错的。如果你已经感觉到饿了，那换一家吧，我怕你撑不到餐来。。。
0,不要辣椒，为什么有呢，不是辣，是不能吃好不好！
0,"感觉还可以吧,味道一般"
0,两个小时才到，饿昏了。不过还是很好吃的。
0,"来这里吃过多次,这次接电话的服务员态度特别差,中午1点多下的单,订4点的外卖,打电话过去,服务员告诉我在那里吃过了,还一副都是我的错的样子,那态度都让人无语了！"
0,酸菜猪骨汤不好喝
0,送餐服务很差
0,太差，，，等了一小时四十分钟没送到，打电话联系不上，，，最糟心的一次。
0,太慢了，就不要了
0,以前很快，不知道这次怎么了，好慢。而且里面没有奥尔良，不知道是没有还是忘记加了，不是很满意。
0,他家没上百度外卖的时候感觉非常好，自从上了百度外卖以后送餐速度大幅下降，原来也就20多分钟就能送到，现在一个小时就算快的，要点大蒜次次忘，这已经是第四次了，饭送来都是凉的，哎等他们过完促销在看看吧，失望至极
0,送的太慢了，11:30左右下的单子，一点才到。都上班了
0,"送餐2个半小时,而且这个米线根本不值28,太不值了"
0,给我忘了三次了，今天又点了，居然都一点多了都没送，打电话都说已经在送了，都没核实是谁就说送了！后来要取消也取消不了，电话也打不通，饭也没给送，也没个说法。害的我中午没吃上饭！气死了！看情况就是把我的忘了的。
0,"1.图片与送餐不一致，套餐也没有说明；,2.米饭太硬；,3.送的晚；"
0,"送餐送的特别特别慢，并且店家电话也打不通、,真是等到抓狂。慢的要死，米线味道还不错，量也足。就是送餐太太太太太太慢、、"
0,送到的和预定是驴头不对马嘴
0,等了一个半小时还不送。电话是空号。呵呵了
0,一个多小时才送到，真是等不起啊
0,我点的凉菜，给我送一盒热的烂白菜
0,和店里点的水平差很多
0,人多有点慢，有点受不了，一等就是两个半小时。
0,蜗牛的速度
0,首先，定的是5点半的，结果4点多就打电话催着拿了。。人都没回来，放那等着凉啊？再者，要求把米线放进去（不放进去味道进不去），拿过来的时候根本很难放进去。。。。PS：送餐员很喜欢挂电话！！！
0,速度太慢了，等了近一个半小时。
0,差！！！！！！！送餐整整用了三个小时！！！一个办公室定了八份，竟然一份一份送！！没有能力接单就不要接，耽误别人的时间！
0,已经第三次给我漏单了，无语
0,单子打错了，少送了。
0,迟到了一个多小时
0,等了136分钟，催了无数次，要不是脚崴了走不了路我早就不要了。
0,送错了还不给换，垃圾！
0,这家店每次都是送餐巨慢！！
0,还是很慢，每次都很慢
0,菜还是不错的，就是实在是太慢！！十一点半点的，等到一点才送！！而且明明点了烤肠也没送
0,太气愤了，送餐送到小区门口，跟我说车没电了，进不来，要去自己拿，态度超级差，还和我嚷！
0,味道一般，送的太晚
0,里边的黄花菜比较少，基本就是黄瓜、肉片和胡萝卜，萝卜吃着还可以，包菜太淡无味，整体感觉一般吧！
0,饭菜味道不错，可能是比较忙吧，送过来较晚，另外就是保温有点差，菜不太热。
0,餐到现在都没送过来，打客服说取消不了；餐未送达，服务态度又不好。
0,这几天送餐越来越慢了，我11点十四分定的餐，将近两个小时才送到，菜品质量也越来越差
0,很好的饺子，可惜者了，怪太慢了
0,等了三个小时，还没有送到，只有退单了！
0,店子不错，但这个菜量太小，不值！
0,坑爹，一点不好吃，想被坑的就来点吧，这种店只会坑我一次，菜一点油都没，这种店注定不会长久。。
0,根本就没送到，距离才1000多米，一个多小时都没有送过来（打电话来），还自己确认收货。。。奸商，百度外卖都没有监管麽。。。
0,菜品口味还是一如既往，经常订餐，但是服务太差，态度极为不好，十点打电话说正在加工，催了三次菜，十一点四十给他打电话竟然说送不过来了，说远这边就我自己订餐，送不过来提前通知啊，我可以订其他的餐啊，希望可以改正，否则我们原先的几个老客不会再去光顾！！
0,虽然很划算，送餐速度也不错，但是不知道送餐的是不是没有保温措施，送来的时候饭菜已经比较凉了。如果愿意出门的话，还是去店里吃比较好。
0,根本就没给我送到，让我等一下午。
0,真的。。。不太好吃。。。大概只吃了十分之一的样子就吃不下去了。。。以后不会再定这个米线了
0,面不够劲道，煮时间偏长？
0,明明卤香鸡腿17元送到后19元，而且鸡腿上有头发。
0,味道不错。菜比较少。最重要的是！！！有一根米线里面有一个小虫子！！！如果米线是卖家自己做的麻烦注意下卫生好嘛！如果是进货买来的，麻烦挑家干净的吧！
0,太慢了。担心无法及时送到，特地提前三小时下单，最后还是延迟了40分钟，而且也没有提前通知，还是我打电话过去问的
0,今天速度慢了些，大概45分钟
0,味道还可以，但是送到时间也太长啦，十一点二十下的单，到下午一点半才送来，打电话打不通，打通就告诉已经出来啦，打了三四遍电话才给从新做了送过来，气愤呀
0,慢的无言以对
0,太久了！河粉汤全都被吸收了！坨了呢！不开心=_=
0,太慢啦！实在难以忍受
0,太黑了，居然涨价
0,好慢啊，等死我了
0,给商家打电话问下外卖的送递的时间，接电话的那个人太霸道了，吼起来了
0,太黑了，涨价了，改地方了
0,菜品质量不错，但是送餐速度太慢了，一个小时才送到，肚子都饿扁了
0,"还有比差评更差的评吗,小迂腐这名儿起的贴切,近两个小时不送,店家还失联,送来了打开一看一度以为送错了,红烧排骨套餐有一块排骨吗,这样的店还有存在的必要吗"
0,人太多就比较慢了，历时两个小时才到
0,一个半小时，太慢了！！！
0,特别慢，不好吃
0,除了速度慢都挺好
0,下次注意，希望不要再弄错了
0,根本不在配送范围，送餐不成功
0,"要的梅菜扣肉给的猪蹄子,囧"
0,晚了，都糟了
0,"米线给的太多了,吃不完"
0,没找到地方自己拿回去了，也不跟顾客说，等了两个小时，服务态度也差劲，千万不能在这家订。。送饭的还理直气壮的，说不关我的事，又不是我送的刚才，饭还是凉的，就让人收。。
0,今天送的慢，说我这边就一个外卖，然后送了快2小时。给客服打电话态度很差！
0,菜没熟，蒜台、青椒没熟。好难吃。
0,为什么我先订餐，同学后二三十分钟订餐，他还能先收到，你们这个是不是搞错了
0,梅干菜扣肉和豆腐丝小菜都特别的咸特别的咸，齁咸，建议少放点盐，梅干菜扣肉如果能带出点甜味也许会中和下齁咸的感觉。相较而言，米饭做的不错，就是量太大，满满一盒饭可以搭配三份菜了。
0,速度有待提高，一个多小时呀
0,等了好久好久，白菜变豆腐丝了，好郁闷
0,酸味还行，量也不错！！送太久了，都凉了。
0,满塑料袋的油，饮料上都是。
0,服务态度还不错，但是今天送货速度太慢了，都饿得发晕了。
0,"还行把,就是饮料到的时候给挤坏了，都撒了。没喝几口"
0,红烧牛肉肉不多。都是青笋
0,和同事一块下的订单，居然同事的先到了，这份晚了40分钟才到，太久了也
0,等了一个多小时，催两次单，不过客服态度还可以。
0,居然漏单，打开一看袋子单点的四喜丸子没送，要点辣椒也没给。太生气了
0,11点下的单，到了下午一点半还没送，两个半小时就算走路也走到了吧，打电话催了3次还不给送，还各种找借口。建议大家不要点他家的外卖，巨坑爹！！！
0,太差劲，等了92分钟居然还没送到，请问有没有职业道德，还讲不讲信誉！
0,巨慢无比，我催单，店家态度还很恶劣
0,丸子不好吃
0,第一次给差评，错开高峰期点餐还送这么慢。可乐直接放在装菜的格子里，可乐瓶上全是油，菜也弄脏了没法吃，饭菜也是凉的。无语，真想退了。
0,太差了，送餐等了一个半小时，送过来餐盒都露了，汤全洒了，大家不要订这的外卖，给店里打电话让我凑活这吃，卧槽，真是窝火
0,这次太慢了
0,木耳太不新鲜，吃完拉肚子了
0,10点50订的，12点半才送来，都凉了
0,一个小时四十分钟送来的。。怎么招都是太慢了。问题是电话也基本打不通，希望改进吧
0,晚了45分钟才送到。。。不过好吃是没挑的
0,说微辣，结果辣死了！
0,送了热汤，但是用普通塑料袋装的，不能喝啊，塑化剂了就！其他的不错
0,送外卖的态度差
0,为什么我的酸辣粉辣么辣
0,和同事一起订的他们都吃完了，我都饿晕了还没给送过来。。。
0,"味道一般,唯一好处,量大,送餐慢"
0,"第一次订还算快,之后越来越慢,这次竟然漏送了一小时后打电话催才开始送,好无奈"
0,说是米饭牛肉套餐居然没米饭只有菜。。我也是醉了。。。这文字功夫下得可真是足啊。。。欺骗消费者
0,肉实在太少了
0,味道不错，送的时间长了点
0,我现在修改一下。就在下午6点8分，我接到了小鱼府客服的电话，她通知我因为今天已经下过一次单了，所以无法再次享受减免活动。在我做出取消订单的决定之后，她说了一句，好，那你今天别吃饭了。我几乎很少给差评。但是这件事不好意思，接下来我要向百度投诉。请小鱼府注意查收
0,速度慢，换的汤太难喝了，咸，炒饼还好吧，总之一般般！
0,比预计送餐时间晚了40分钟，餐食也凉了，吃的不舒服。关键是师傅送的第一单地址离我这儿很近只有几百米，但是他选择无视，直接略过，在望京兜了一大圈直到我和客服联系催餐之后才送来。知道中午订单多，送餐师傅辛苦。但是合理规划路径很重要啊！！！
0,送货员很好，速度快态度好。螺狮粉实在…有股臭臭的味道，吃完胃难受了一晚上，吐出来才好受。
0,为什么会有一股冲天的腐臭味!那么多好评是水军刷出来的吗
0,"别的百度骑士都给送上楼,这个没有给送到楼上,没有就算了,态度也一般,就这样吧,粉吃起来的味道还是可以的,气味稍微有一点怪。。而且并没有见到螺丝,有点失望"
0,说好的加辣呢？知春里套餐送的饮料呢？我点的猪脚呢？订的外卖少了一大半！！！这配送素质真是给跪了！！
0,这次的凉了，很难吃，短期内不会再点
0,很难吃，还吃到紫色的塑料东西，恶心
0,送错了东西还有理了啊～错了就是错了，打电话告诉你们，你们道个歉就完了，女的接的电话不道歉态度特别恶略，很还这那那这的～实在是太恶心了！！！！！！
0,无语了，买四份豆泡来了只有一份，沟通之后就没消息了
0,"不好吃,粉,感觉不对,不够臭,哈哈哈"
0,辣得不正。快递催了两次。
0,点的中辣，送来的次次都是微辣，服了，难道每次都会看错么？
0,还真是水平有限
0,不知道什么味，特难闻，吃一口就都扔了，和包装袋有关系吧，哎
0,"怎么一股汽油味？难吃死了,食材也太差了,浪费钱"
0,送早了，米粉成坨坨了。。。
0,太差劲了！！！
0,朋友说今天的菜不新鲜还吃出了头发
0,没给发票啊~
0,我要的是中辣，给我送的是一点儿辣椒都没有！！！！
0,"送来的瞬间我就不想吃了,送餐员身上的味道太大了,唉,我都不好意思说"
0,要了俩卤蛋，送来才一个
0,什么叫小葱没有了…只给了一包，白醋也是一包，两个人根本不够好么？为何越来越小气？
0,12:30配送员给我电话告诉我他车子和餐都丢了！让我取消订单…我取消不了，电话百度外卖，客服说可以取消，会帮我操作。于是另外定了餐，正吃着呢，百度又来送餐了！不知道什么环节没沟通好，导致我今天中午面对两份外卖…呜呜呜，很差的体验是给百度外卖的！米粉很好吃！配送员态度也很好！
0,粉味道还可以，但是猪手居然是臭的，尼玛，臭的！皮还咬不动，能有点职业操守么
0,光觉得辣了，绿豆沙也比较一般
0,汤里有虫子，此生不买第二回
0,"等了2小时终于送到了。,然后订的青菜黄了，绿豆沙也撒了，汤也冷了，粉硬到拌不开。,有个超时赔付有什么用呢，能吃吗？还不是得倒掉。"
0,粉送來的時候已經「斷」完了，跟米飯一樣，唉，有些失望⋯
0,2点才送来！！
0,"汤还可以,粉一般,上次在店里吃就觉得好像不熟,这次外卖还没在店里好吃"
0,真的两个多小时！！！！
0,汤，粉和菜品应该用正规的餐盒装着，担心漏出来就在外面包上保鲜膜，其他家都这么做的。
0,差评，有送餐费还没饿了吗没有送餐费快。最后两分钟送到，这可是米粉不是米饭都已经没法吃了，你们自己吃吃看放了一个半小时的米粉会是怎样？
0,送餐很慢，等了一个多小时，竟然还送错了，餐厅和配送员都没有核对单子，价钱都差好多，最后让我对付吃吧，要不然就再等一个小时⋯⋯
0,等了好久才送到~
0,螺蛳粉还是不错的，绿豆沙也很好喝，但是这送餐速度也太让人无语了吧，不说自己不认识路，还怪我们把地址写错，气人！
0,忍住没给一星……少送了鸭脚，店家承诺下次补送。粉里有毛发，眉毛那么长的，特别影响心情……希望能注意，谢了。
0,送的太慢啦，餐还没到呢订单怎么就完成啦
0,我十一点一刻点的餐，四点了才到。中饭变成了晚餐！！！！第一次！！！也是最后一次！！！不说了！！！都是泪！！！
0,就是时间稍微长些
0,我只想说，真的很慢，我们公司就在它对面，可是时间还是真的慢11点半订的餐，13点半才送到
0,我想说的是，正如评价说的，一点都不好吃！一送来，整个家都臭了，还有那个酸笋太难吃了！！！以前在桂林米粉吃的超好吃！这个一点酸味没有，而且感觉快坏了
0,没送到擅自点击送餐完成，近几次定餐配送都在一个半小时左右。
0,关于送餐，速度我已经无力吐槽了！！！一个半小时，一个半小时，一个半小时……
0,不喜欢这个口味儿
0,口味一般！蕨菜用塑料袋装的，感觉很不好，为什么不用小碗装呢？以后不会吃了
0,时间过长，最后也没有得到5折优惠啊！
0,单加的青菜里有根头发
0,点的超辣，结果是不辣的，而且等了一个多小时才到。真忍了。
0,完了两个小时，不能接受
0,水平有限没问题，但不应该少给东西啊！俺单点的鸭肠呢？
0,"太臭了,真的"
0,"快递小哥坚持送到位，表扬一下。,螺狮粉味道太一般。"
0,11点以前就定的餐，做了1小时48分钟，呵呵，我只想说：拜拜！！！
0,"外卖骑士将近两小时送到,而且还要我下去取餐"
0,"味道好一般哦,配料不丰富"
0,"恶心…要吐了…直接扔了,一万年才到。"
0,送了两个多小时！大家吃饭我工作，大家开工了，我端一碗臭臭的螺狮粉，想想什么感受吧！快递大哥态度很好不停道歉，只能说百度系统流程都太烂，不规划好路线两个小时十单都送不完，就是这结果。说说菜品，粉不错，鸭肠不好一股没处理好的味儿，鸭掌还行，绿豆汤送到都撒了一半。
0,少一个卤蛋
0,绿豆汤馊了！！！！！！我明天铁定拉肚子！
0,味道太不咋的了
0,"我能说我没有吃过这么。。。。算了一人一口味,等了一个多小时真的太失望了。第一次吃也是最后一次了。。。"
0,味道依然很好，送餐等了一个半小时…
0,"吃出一个毛发,和一堆不知道是什么的东西。。"
0,一个半小时送到，粉都砣在一起了，很无语
0,除了时间慢没别的不好
0,速度满，十一点四十下的单，一点半到的菜叶都是黄的，我也是醉了，送餐员满头大汗，我是心疼了
0,家里人说有股臭臭的味道~
0,送餐时间还是比较久
0,下了单说一个小时之后送，过了一个小时送餐员打电话又说晚15分钟，而且态度不好！味道也一般，跟小螺号比差远了，再也不点了
0,送餐太慢，导致粉都坨了
0,味道没店里的好。订单少送一个绿豆沙，发票的单子的价格也是错的，懒得再打电话，送了1个小时20分钟才送到，再也不吃他家外卖了
0,少了绿豆沙，最近是怎么了，钱都网上付了可老是少东西，外卖送来的的点菜单上就没打上绿豆沙
0,"太难吃,点了绿豆沙也没给我。。。坑"
0,粉很好吃，但是订了卤蛋却没有。
0,送了快两个钟头才到……
0,粉很好吃，就是配送员找不到位置还一直埋怨，送来时已经坨了！
0,"排骨不好吃～还都是色素～量少的可怜～58元的菜啊,只有七块排骨～一点味道没有～肉很干～坑死了！还招牌菜呢！真不怕砸了招牌！"
0,饺子本来味道不错，也不是第一次订了，但是这次这包装弄成这样太气人了，一种馅的饺子我要了好几份，竟然全都装在了一个盒里挤着，到我这的时候饺子都被挤的分不开了，拿筷子一夹就烂了，倒出来以后就是一个包装盒的形状，分都分不开，就不能多装一个盒子么，本来饺子这种东西就容易陀在一起，这么多饺子还挤在一个盒里，到了我这还能吃么
0,肉越来越少了，说是纯瘦，其实也有肥肉
0,送餐速度太慢了，几百米将近半个小时才送到，油泼面都没法吃了……很不满意
0,油泼面没熟，都是粘的，坨一大疙瘩了。
0,你们的菜品质量已经刷新新的下降高度，店家你们知道吗？这么好的地方美食你们忍心做成这样吗？
0,电话投诉三次没人理！！！百度服务太差！！！系统错误也没个说话，明明点了两份就送一份，半个月了，每次都说尽快回复，至今没接到过百度电话！！！
0,麻辣羊血不好吃
0,谢谢！速度很快辛苦了！
0,和以前的大不一样了，不怎么好吃
0,满三十为啥没优惠了。
0,连双筷子都没有，明明点的葡萄汁结果送来的是酸梅汤，也真是无语了！
0,不吃辣，都给的我们辣
0,鸡蛋都坏了，凉菜也洒了
0,差的无法形容，我花18元买的羊肉汤，结果拿来里面就一块肉。粉丝也就那么几根。从来没见过这么坑爹的商家了，简直像别人吃剩下的给我送来的？恶心的要命，卤蛋也是不新鲜的？这个好在没几元钱啊！等倒闭吧你们
0,外送员很赞，商家能不能仔细看订单啊！点的干拌面送来的是汤面，说了粉汤羊血要多加辣椒送来的一点儿辣没有！！
0,味道可以试试
0,不怎么地'，凉的，吃完恶心
0,"羊汤也太次了,哪有放油菜的,服了"
0,肉夹馍没送来，告诉我十分钟内送到，可是二十分钟了还没到
0,肉夹馍肉太少
0,以前几乎天天吃，现在调料什么都不放，
0,昨天订凉皮两份，什么调料都没有放，就放了点麻油，特别难吃，丢了一份，再也不想吃了
0,"凉皮太辣,吃不下都"
0,本来迟到了还自己点！！！'
        >>>senti.sentiment_count(test_text)
        >>>{'words': 85479, 'sentences': 1000, 'pos': 894, 'neg': 588}

    考虑强度副词(如"非常"，"差不多""太")对情感形容词的修饰作用，
    和否定词对情感意义的反转作用。
    返回的pos和neg是得分
        >>>senti.sentiment_calculate(test_text)
        >>>{'sentences': 1000, 'words': 85479, 'pos': 38.0, 'neg': 62.0}


    使用自定义txt词典(建议utf-8编码)，
    目前仅支持pos和neg两类词典，每行一个词语。
    merge=True，cnsenti会融合自带的词典和用户导入的自定义词典；merge=False，cnsenti只使导入的自定义词典
    其中pos和neg为txt词典文件路径，encoding为txt词典的编码方式
    这里是utf-8编码的文件，初始化方式
        >>>from cnsenti import Sentiment
        >>>senti = Sentiment(pos='正面词典.txt', neg='负面词典.txt', merge=True, encoding='utf-8')
    """

    def __init__(self, merge=True, pos=None, neg=None, encoding='utf-8'):
        """
        :pos 正面词典的txt文件
        :neg 负面词典的txt文件
        :merge 默认merge=True,即融合自带情感词典和自定义词典。merge=False，只使用自定义词典。
        :encoding 词典txt文件的编码，默认为utf-8。如果是其他编码，该参数必须使用
        """
        self.Poss = self.load_dict('pos.pkl')
        self.Negs = self.load_dict('neg.pkl')

        if pos:
            if merge:
                del self.Poss
                self.Poss = self.load_diydict(file=pos, encoding=encoding)+self.load_dict('pos.pkl')
                jieba.load_userdict(pos)

            else:
                del self.Poss
                self.Poss = self.load_diydict(file=pos, encoding=encoding)
                jieba.load_userdict(pos)


        if neg:
            if merge:
                del self.Negs
                self.Negs = self.load_diydict(file=neg, encoding=encoding)+self.load_dict('neg.pkl')
                jieba.load_userdict(neg)
            else:
                del self.Negs
                self.Negs = self.load_diydict(file=neg, encoding=encoding)
                jieba.load_userdict(neg)

        self.Denys = self.load_dict('deny.pkl')

        self.Extremes = self.load_dict('extreme.pkl')
        self.Verys = self.load_dict('very.pkl')
        self.Mores = self.load_dict('more.pkl')
        self.Ishs = self.load_dict('ish.pkl')

    def load_dict(self, file):
        """
        Sentiment内置的读取hownet自带pkl词典
        :param file:  词典pkl文件
        :return: 词语列表
        """
        #pathchain = ['dictionary', 'hownet',file]
        pathchain = ['hownet',file]
        mood_dict_filepath = pathlib.Path('__file__').parent.joinpath(*pathchain)
        dict_f = open(mood_dict_filepath, 'rb')
        words = pickle.load(dict_f)
        return words

    def load_diydict(self, file, encoding):
        """
        :param file:  自定义txt情感词典，其中txt文件每行只能放一个词
        :param encoding:  txt文件的编码方式
        :return:
        """
        text = open(file, encoding=encoding).read()
        words = text.split('\n')
        words = [w for w in words if w]
        return words


    def sentiment_count(self, text):
        """
        简单情感分析，未考虑强度副词、否定词对情感的复杂影响。仅仅计算各个情绪词出现次数(占比)
        :param text:  中文文本字符串
        :return: 返回情感信息，形如{'sentences': 2, 'words': 24, 'pos': 46.0, 'neg': 0.0}
        """
        length, sentences, pos, neg = 0, 0, 0, 0
        sentences = [s for s in re.split('[\.。！!？\?\n;；]+', text) if s]
        sentences = len(sentences)
        words = jieba.lcut(text)
        length = len(words)
        for w in words:
            if w in self.Poss:
                pos+=1
            elif w in self.Negs:
                neg+=1
            else:
                pass
        return {'words': length,  'sentences':sentences, 'pos':pos, 'neg':neg}


    def judgeodd(self, num):
        """
        判断奇数偶数。当情感词前方有偶数个否定词，情感极性方向不变。奇数会改变情感极性方向。
        """
        if (num % 2) == 0:
            return 'even'
        else:
            return 'odd'

    def sentiment_calculate(self, text):
        """
        考虑副词对情绪形容词的修饰作用和否定词的反转作用，
        其中副词对情感形容词的情感赋以权重，
        否定词确定情感值正负。

        :param text:  文本字符串
        :return: 返回情感信息，刑如{'sentences': 2, 'words': 24, 'pos': 46.0, 'neg': 0.0}
        """
        sentences = [s for s in re.split('[\.。！!？\?\n;；]+', text) if s]
        wordnum = len(jieba.lcut(text))
        count1 = []
        count2 = []
        for sen in sentences:
            segtmp = jieba.lcut(sen)
            i = 0  # 记录扫描到的词的位置
            a = 0  # 记录情感词的位置
            poscount = 0  # 积极词的第一次分值
            poscount2 = 0  # 积极词反转后的分值
            poscount3 = 0  # 积极词的最后分值（包括叹号的分值）
            negcount = 0
            negcount2 = 0
            negcount3 = 0
            for word in segtmp:
                if word in self.Poss:  # 判断词语是否是情感词
                    poscount += 1
                    c = 0
                    for w in segtmp[a:i]:  # 扫描情感词前的程度词
                        if w in self.Extremes:
                            poscount *= 4.0
                        elif w in self.Verys:
                            poscount *= 3.0
                        elif w in self.Mores:
                            poscount *= 2.0
                        elif w in self.Ishs:
                            poscount *= 0.5
                        elif w in self.Denys:
                            c += 1
                    if self.judgeodd(c) == 'odd':  # 扫描情感词前的否定词数
                        poscount *= -1.0
                        poscount2 += poscount
                        poscount = 0
                        poscount3 = poscount + poscount2 + poscount3
                        poscount2 = 0
                    else:
                        poscount3 = poscount + poscount2 + poscount3
                        poscount = 0
                    a = i + 1  # 情感词的位置变化

                elif word in self.Negs:  # 消极情感的分析，与上面一致
                    negcount += 1
                    d = 0
                    for w in segtmp[a:i]:
                        if w in self.Extremes:
                            negcount *= 4.0
                        elif w in self.Verys:
                            negcount *= 3.0
                        elif w in self.Mores:
                            negcount *= 2.0
                        elif w in self.Ishs:
                            negcount *= 0.5
                        elif w in self.Denys:
                            d += 1
                    if self.judgeodd(d) == 'odd':
                        negcount *= -1.0
                        negcount2 += negcount
                        negcount = 0
                        negcount3 = negcount + negcount2 + negcount3
                        negcount2 = 0
                    else:
                        negcount3 = negcount + negcount2 + negcount3
                        negcount = 0
                    a = i + 1
                elif word == '！' or word == '!':  ##判断句子是否有感叹号
                    for w2 in segtmp[::-1]:  # 扫描感叹号前的情感词，发现后权值+2，然后退出循环
                        if w2 in self.Poss or self.Negs:
                            poscount3 += 2
                            negcount3 += 2
                            break
                i += 1  # 扫描词位置前移

                # 以下是防止出现负数的情况
                pos_count = 0
                neg_count = 0
                if poscount3 < 0 and negcount3 > 0:
                    neg_count += negcount3 - poscount3
                    pos_count = 0
                elif negcount3 < 0 and poscount3 > 0:
                    pos_count = poscount3 - negcount3
                    neg_count = 0
                elif poscount3 < 0 and negcount3 < 0:
                    neg_count = -poscount3
                    pos_count = -negcount3
                else:
                    pos_count = poscount3
                    neg_count = negcount3

                count1.append([pos_count, neg_count])
            count2.append(count1)
            count1 = []

        pos_result = []
        neg_result = []
        for sentence in count2:
            score_array = np.array(sentence)
            pos = np.sum(score_array[:, 0])
            neg = np.sum(score_array[:, 1])
            pos_result.append(pos)
            neg_result.append(neg)

        pos_score = np.sum(np.array(pos_result))
        neg_score = np.sum(np.array(neg_result))
        score = {'sentences': len(count2),
                 'words':wordnum,
                 'pos': pos_score,
                 'neg': neg_score}

        return score
    


<>:1109: SyntaxWarning: invalid escape sequence '\.'
<>:1141: SyntaxWarning: invalid escape sequence '\.'
<>:1109: SyntaxWarning: invalid escape sequence '\.'
<>:1141: SyntaxWarning: invalid escape sequence '\.'
C:\Users\ASUS\AppData\Local\Temp\ipykernel_25868\3746984806.py:1109: SyntaxWarning: invalid escape sequence '\.'
  sentences = [s for s in re.split('[\.。！!？\?\n;；]+', text) if s]
C:\Users\ASUS\AppData\Local\Temp\ipykernel_25868\3746984806.py:1141: SyntaxWarning: invalid escape sequence '\.'
  sentences = [s for s in re.split('[\.。！!？\?\n;；]+', text) if s]


In [10]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, recall_score, f1_score, confusion_matrix

# 假设df是包含情感标签和评论的Pandas DataFrame
# df['label']是情感标签（1为正面，0为负面）
# df['review']是评论文本

# 分割数据集
X_train, X_test, y_train, y_test = train_test_split(df['review'], df['label'], test_size=0.2, random_state=42)

# 特征提取
vectorizer = TfidfVectorizer()
X_train_vectors = vectorizer.fit_transform(X_train)
X_test_vectors = vectorizer.transform(X_test)

# 模型训练
model = MultinomialNB()
model.fit(X_train_vectors, y_train)

# 模型预测
y_pred = model.predict(X_test_vectors)

# 计算性能指标
accuracy = accuracy_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

# 打印性能指标
print(f"Accuracy: {accuracy}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

ModuleNotFoundError: No module named 'sklearn'